Задание 6. Частичная проблема собственных значений — поиск максимального по модулю собственного числа степенным методом и методом скалярных произведений.

In [1]:
import numpy as np
import pandas as pd
from typing import Optional, Tuple
from numpy import linalg as LA
from math import sqrt
from scipy.linalg import hilbert

In [2]:
def power_method(matrix: np.ndarray, accuracy: float, limit=10000) -> Tuple[float | None, Optional[np.array], int]:
    current_x = np.ones(matrix.shape[1])

    for step in range(1, limit):
        next_x = matrix @ current_x
        
        value = sqrt(np.dot(next_x, next_x) / np.dot(current_x, current_x)) * np.sign(next_x[0] / current_x[0])
        if LA.norm(next_x - value * current_x) / LA.norm(current_x) <= accuracy:
            return value, next_x, step

        current_x = next_x / LA.norm(next_x)

    return None, None, step

In [3]:
def scalar_product_method(matrix: np.ndarray, accuracy: float, limit=10000) -> Tuple[float | None, Optional[np.array], int]:
    size = matrix.shape[1]
    current_x, current_y = np.ones(size), np.ones(size)
    transposed_matrix = np.transpose(matrix)
    current_value = None

    for step in range(limit):
        previous_x = current_x
        current_x, current_y = matrix @ current_x, transposed_matrix @ current_y
        current_y = current_y / LA.norm(current_y)

        previous_value = current_value
        current_value = np.dot(current_x, current_y) / np.dot(previous_x, current_y)
        if previous_value is not None and abs(current_value - previous_value) <= accuracy:
            return current_value, current_x, step

        current_x = current_x / LA.norm(current_x)
        

    return None, None, step

In [4]:
methods = {
    "степенной метод": power_method,
    "метод скалярных произведений": scalar_product_method
}

columns = ["точность", "метод", "количество итераций",
           "вычисленное собственное число", "погрешность"]


def display_results(matrix: np.ndarray):
    values, _ = LA.eig(matrix)
    exact_value = max(values, key=lambda v: abs(v)).real
    print(f'"Точное" собственное число: {exact_value}')

    accuracy = [10 ** -p for p in range(2, 6)]
    data = []

    for a in accuracy:
        for m_name, m in methods.items():
            actual_value, _, steps = m(matrix, a)
            data.append([a, m_name, steps, actual_value,
                        abs(exact_value - actual_value)])

    df = pd.DataFrame(data, columns=columns)
    display(df.style.format(
        {columns[0]: '{:.1e}',
         columns[4]: '{:.6e}',
         columns[3]: '{:.16f}'}
    ).set_table_styles(
        [{'selector': 'th', 'props': [('max-width', '100px')]}]
    ).hide())

### Диагональная матрица

In [5]:
display_results(np.diag(range(-5, 0)))

"Точное" собственное число: -5.0


точность,метод,количество итераций,вычисленное собственное число,погрешность
1.0e-02,степенной метод,22,-4.9999234416242091,7.655838e-05
1.0e-02,метод скалярных произведений,9,-4.9856738615653162,1.432614e-02
1.0e-03,степенной метод,32,-4.9999991172833278,8.827167e-07
1.0e-03,метод скалярных произведений,14,-4.9984542305610846,1.545769e-03
1.0e-04,степенной метод,43,-4.9999999934866945,6.513305e-09
1.0e-04,метод скалярных произведений,19,-4.9998338696482039,1.661304e-04
1.0e-05,степенной метод,53,-4.9999999999249063,7.509371e-11
1.0e-05,метод скалярных произведений,25,-4.9999885821391281,1.141786e-05


### Симметричная матрица

In [6]:
display_results(np.array([[-1.48213, -0.03916, 1.08254],
                          [-0.03916, 1.13958, 0.01617],
                          [1.08254, 0.01617, -1.48271]]))


"Точное" собственное число: -2.5653731572905434


точность,метод,количество итераций,вычисленное собственное число,погрешность
1.0e-02,степенной метод,14,-2.5653668222411139,6.335049e-06
1.0e-02,метод скалярных произведений,10,-2.5666887549192512,1.315598e-03
1.0e-03,степенной метод,17,-2.5653731084641547,4.882639e-08
1.0e-03,метод скалярных произведений,12,-2.5654244721634720,5.131487e-05
1.0e-04,степенной метод,20,-2.5653731569142249,3.763185e-10
1.0e-04,метод скалярных произведений,13,-2.5653832934303877,1.013614e-05
1.0e-05,степенной метод,23,-2.5653731572876439,2.899458e-12
1.0e-05,метод скалярных произведений,14,-2.5653751594829162,2.002192e-06


### Матрица Гильберта

In [7]:
display_results(hilbert(10))

"Точное" собственное число: 1.7519196702651776


точность,метод,количество итераций,вычисленное собственное число,погрешность
1.0e-02,степенной метод,4,1.7519027265035705,1.694376e-05
1.0e-02,метод скалярных произведений,2,1.7517749020467632,1.447682e-04
1.0e-03,степенной метод,6,1.7519196453892509,2.487593e-08
1.0e-03,метод скалярных произведений,3,1.7519141227712851,5.547494e-06
1.0e-04,степенной метод,7,1.7519196693120285,9.531491e-10
1.0e-04,метод скалярных произведений,4,1.7519194577059773,2.125592e-07
1.0e-05,степенной метод,8,1.7519196702286566,3.652101e-11
1.0e-05,метод скалярных произведений,4,1.7519194577059773,2.125592e-07
